In [4]:
import mysql.connector

# Establishing a connection to the database
conn = mysql.connector.connect(
    host="localhost",           # Hostname of your MySQL server
    user="root",                # Your MySQL username
    password="TaNu@170603",     # Your MySQL password
    database="test",            # Name of the database to use
    port=3307                   # MySQL port (if not default 3306)
)

# Creating a cursor object to interact with the database
cursor = conn.cursor()

# SQL query to insert data into the employees table
sql = "INSERT INTO employees (name, age, position, hire_date) VALUES (%s, %s, %s, %s)"

# Values to insert into the table
values = ("yash Doe", 30, "Software Engineer", "2024-12-15")

# Executing the SQL query
cursor.execute(sql, values)

# Committing the changes to the database
conn.commit()

# Printing success message
print("Data inserted successfully")

# Closing the cursor and connection
cursor.close()
conn.close()


Data inserted successfully


In [ ]:
from flask import Flask, render_template, request, redirect, url_for, flash
import mysql.connector
from werkzeug.security import generate_password_hash, check_password_hash

app = Flask(__name__)
app.secret_key = 'your_secret_key'  # Required for flash messages

# Database connection configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'TaNu@170603',
    'database': 'test',
    'port': 3307
}

# Home route
@app.route('/')
def home():
    return render_template('home.html')

# Sign-up route
@app.route('/signup', methods=['GET', 'POST'])
def signup():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        hashed_password = generate_password_hash(password, method='sha256')

        # Insert user into the database
        try:
            conn = mysql.connector.connect(**db_config)
            cursor = conn.cursor()
            cursor.execute("INSERT INTO users (username, password) VALUES (%s, %s)", (username, hashed_password))
            conn.commit()
            flash('User registered successfully! Please log in.', 'success')
        except mysql.connector.Error as err:
            flash(f"Error: {err}", 'danger')
        finally:
            if conn.is_connected():
                cursor.close()
                conn.close()

        return redirect(url_for('home'))

    return render_template('signup.html')

# Login route
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        # Verify user in the database
        try:
            conn = mysql.connector.connect(**db_config)
            cursor = conn.cursor()
            cursor.execute("SELECT password FROM users WHERE username = %s", (username,))
            user = cursor.fetchone()

            if user and check_password_hash(user[0], password):
                flash('Login successful!', 'success')
            else:
                flash('Invalid username or password.', 'danger')
        except mysql.connector.Error as err:
            flash(f"Error: {err}", 'danger')
        finally:
            if conn.is_connected():
                cursor.close()
                conn.close()

        return redirect(url_for('home'))

    return render_template('login.html')

if __name__ == '__main__':
    app.run(debug=True)
